In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier

In [ ]:
data = pd.read_csv('Airbnb_Data.csv')

In [ ]:
data.head()

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,...,latitude,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds
0,6901257,5.010635,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",3,1.0,Real Bed,strict,True,...,40.696524,-73.991617,Beautiful brownstone 1-bedroom,Brooklyn Heights,2,100.0,https://a0.muscache.com/im/pictures/6d7cbbf7-c...,11201,1.0,1.0
1,6304928,5.129899,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",7,1.0,Real Bed,strict,True,...,40.766115,-73.989040,Superb 3BR Apt Located Near Times Square,Hell's Kitchen,6,93.0,https://a0.muscache.com/im/pictures/348a55fe-4...,10019,3.0,3.0
2,7919400,4.976734,Apartment,Entire home/apt,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",5,1.0,Real Bed,moderate,True,...,40.808110,-73.943756,The Garden Oasis,Harlem,10,92.0,https://a0.muscache.com/im/pictures/6fae5362-9...,10027,1.0,3.0
3,13418779,6.620073,House,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",4,1.0,Real Bed,flexible,True,...,37.772004,-122.431619,Beautiful Flat in the Heart of SF!,Lower Haight,0,NaN,https://a0.muscache.com/im/pictures/72208dad-9...,94117.0,2.0,2.0
4,3808709,4.744932,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.0,Real Bed,moderate,True,...,38.925627,-77.034596,Great studio in midtown DC,Columbia Heights,4,40.0,NaN,20009,0.0,1.0


In [ ]:
#

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74111 entries, 0 to 74110
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      74111 non-null  int64  
 1   log_price               74111 non-null  float64
 2   property_type           74111 non-null  object 
 3   room_type               74111 non-null  object 
 4   amenities               74111 non-null  object 
 5   accommodates            74111 non-null  int64  
 6   bathrooms               73911 non-null  float64
 7   bed_type                74111 non-null  object 
 8   cancellation_policy     74111 non-null  object 
 9   cleaning_fee            74111 non-null  bool   
 10  city                    74111 non-null  object 
 11  description             74111 non-null  object 
 12  first_review            58247 non-null  object 
 13  host_has_profile_pic    73923 non-null  object 
 14  host_identity_verified  73923 non-null

In [ ]:
data.describe()

,id,log_price,accommodates,bathrooms,latitude,longitude,number_of_reviews,review_scores_rating,bedrooms,beds
count,7.411100e+04,74111.000000,74111.000000,73911.000000,74111.000000,74111.000000,74111.000000,57389.000000,74020.000000,73980.000000
mean,1.126662e+07,4.782069,3.155146,1.235263,38.445958,-92.397525,20.900568,94.067365,1.265793,1.710868
std,6.081735e+06,0.717394,2.153589,0.582044,3.080167,21.705322,37.828641,7.836556,0.852143,1.254142
min,3.440000e+02,0.000000,1.000000,0.000000,33.338905,-122.511500,0.000000,20.000000,0.000000,0.000000
25%,6.261964e+06,4.317488,2.000000,1.000000,34.127908,-118.342374,1.000000,92.000000,1.000000,1.000000
50%,1.225415e+07,4.709530,2.000000,1.000000,40.662138,-76.996965,6.000000,96.000000,1.000000,1.000000
75%,1.640226e+07,5.220356,4.000000,1.000000,40.746096,-73.954660,23.000000,100.000000,1.000000,2.000000
max,2.123090e+07,7.600402,16.000000,8.000000,42.390437,-70.985047,605.000000,100.000000,10.000000,18.000000


In [ ]:
data.isnull().sum()

,0
id,0
log_price,0
property_type,0
room_type,0
amenities,0
accommodates,0
bathrooms,200
bed_type,0
cancellation_policy,0
cleaning_fee,0


In [ ]:
airbnb = data.drop(['id', 'description','first_review', 'host_has_profile_pic',
                    'host_since', 'last_review', 'latitude', 'longitude', 'name',
                    'neighbourhood','thumbnail_url', 'zipcode'], axis = 1)
airbnb.head()

,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,city,host_identity_verified,host_response_rate,instant_bookable,number_of_reviews,review_scores_rating,bedrooms,beds
0,5.010635,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",3,1.0,Real Bed,strict,True,NYC,t,NaN,f,2,100.0,1.0,1.0
1,5.129899,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",7,1.0,Real Bed,strict,True,NYC,f,100%,t,6,93.0,3.0,3.0
2,4.976734,Apartment,Entire home/apt,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",5,1.0,Real Bed,moderate,True,NYC,t,100%,t,10,92.0,1.0,3.0
3,6.620073,House,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",4,1.0,Real Bed,flexible,True,SF,t,NaN,f,0,NaN,2.0,2.0
4,4.744932,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.0,Real Bed,moderate,True,DC,t,100%,t,4,40.0,0.0,1.0


In [ ]:
airbnb.shape

(74111, 17)

In [ ]:
airbnb.isnull().sum()

,0
log_price,0
property_type,0
room_type,0
amenities,0
accommodates,0
bathrooms,200
bed_type,0
cancellation_policy,0
cleaning_fee,0
city,0


In [ ]:
# Impute numerical missing values with median
airbnb['bathrooms'].fillna(airbnb['bathrooms'].median(), inplace=True)
airbnb['review_scores_rating'].fillna(airbnb['review_scores_rating'].median(), inplace=True)
airbnb['bedrooms'].fillna(airbnb['bedrooms'].median(), inplace=True)
airbnb['beds'].fillna(airbnb['beds'].median(), inplace=True)

# Impute categorical missing values with the most frequent value
airbnb['cleaning_fee'].fillna(airbnb['cleaning_fee'].mode()[0], inplace=True)
airbnb['instant_bookable'].fillna(airbnb['instant_bookable'].mode()[0], inplace=True)

<ipython-input-34-ebbbd939c403>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  airbnb['bathrooms'].fillna(airbnb['bathrooms'].median(), inplace=True)
<ipython-input-34-ebbbd939c403>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, in

In [ ]:
airbnb.isnull().sum()

,0
log_price,0
property_type,0
room_type,0
amenities,0
accommodates,0
bathrooms,0
bed_type,0
cancellation_policy,0
cleaning_fee,0
city,0


In [ ]:
airbnb.shape

(74111, 17)

In [ ]:
# itterate over catigorical column and print value counts for each column
for col in airbnb.select_dtypes(include=['object']):
    print(airbnb[col].value_counts())

property_type
Apartment             49003
House                 16511
Condominium            2658
Townhouse              1692
Loft                   1244
Other                   607
Guesthouse              498
Bed & Breakfast         462
Bungalow                366
Villa                   179
Dorm                    142
Guest suite             123
Camper/RV                94
Timeshare                77
Cabin                    72
In-law                   71
Hostel                   70
Boutique hotel           69
Boat                     65
Serviced apartment       21
Tent                     18
Castle                   13
Vacation home            11
Yurt                      9
Hut                       8
Treehouse                 7
Chalet                    6
Earth House               4
Tipi                      3
Train                     2
Cave                      2
Casa particular           1
Parking Space             1
Lighthouse                1
Island                    1
Name: 

In [ ]:
# One-hot encoding for categorical variables
airbnb = pd.get_dummies(airbnb, drop_first=True)
airbnb.head()

,log_price,accommodates,bathrooms,cleaning_fee,number_of_reviews,review_scores_rating,bedrooms,beds,property_type_Bed & Breakfast,property_type_Boat,...,host_response_rate_91%,host_response_rate_92%,host_response_rate_93%,host_response_rate_94%,host_response_rate_95%,host_response_rate_96%,host_response_rate_97%,host_response_rate_98%,host_response_rate_99%,instant_bookable_t
0,5.010635,3,1.0,True,2.0,100.0,1.0,1.0,False,False,...,False,False,False,False,False,False,False,False,False,False
1,5.129899,7,1.0,True,6.0,93.0,3.0,3.0,False,False,...,False,False,False,False,False,False,False,False,False,True
2,4.976734,5,1.0,True,10.0,92.0,1.0,3.0,False,False,...,False,False,False,False,False,False,False,False,False,True
3,6.620073,4,1.0,True,0.0,96.0,2.0,2.0,False,False,...,False,False,False,False,False,False,False,False,False,False
4,4.744932,2,1.0,True,4.0,40.0,0.0,1.0,False,False,...,False,False,False,False,False,False,False,False,False,True


In [ ]:
airbnb.isnull().sum()

,0
log_price,0
accommodates,0
bathrooms,0
cleaning_fee,0
number_of_reviews,1
...,...
host_response_rate_96%,0
host_response_rate_97%,0
host_response_rate_98%,0
host_response_rate_99%,0


In [ ]:
airbnb.head()

,log_price,accommodates,bathrooms,cleaning_fee,number_of_reviews,review_scores_rating,bedrooms,beds,property_type_Bed & Breakfast,property_type_Boat,...,host_response_rate_91%,host_response_rate_92%,host_response_rate_93%,host_response_rate_94%,host_response_rate_95%,host_response_rate_96%,host_response_rate_97%,host_response_rate_98%,host_response_rate_99%,instant_bookable_t
0,5.010635,3,1.0,True,2.0,100.0,1.0,1.0,False,False,...,False,False,False,False,False,False,False,False,False,False
1,5.129899,7,1.0,True,6.0,93.0,3.0,3.0,False,False,...,False,False,False,False,False,False,False,False,False,True
2,4.976734,5,1.0,True,10.0,92.0,1.0,3.0,False,False,...,False,False,False,False,False,False,False,False,False,True
3,6.620073,4,1.0,True,0.0,96.0,2.0,2.0,False,False,...,False,False,False,False,False,False,False,False,False,False
4,4.744932,2,1.0,True,4.0,40.0,0.0,1.0,False,False,...,False,False,False,False,False,False,False,False,False,True


In [ ]:
airbnb.iloc[:20000,:].shape

(9936, 9736)

In [ ]:
X = airbnb.drop('log_price', axis=1)
y = airbnb['log_price']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [ ]:
# Define the neural network model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1), activation = 'sigmoid')  # Output layer for regression

In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

In [ ]:
# Predict on the test set
y_pred = model.predict(X_test)

# Instantiate the Mean Absolute Error metric
mae_metric = tf.keras.metrics.MeanAbsoluteError()

# Update the metric with the true and predicted values
mae_metric.update_state(y_test, y_pred)

# Calculate and print the Mean Absolute Error
mae_value = mae_metric.result().numpy()  # Convert the result to a numpy value

print("Mean Absolute Error:", mae_value)

In [ ]:
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()